In [0]:
import json,os,re
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, ArrayType, LongType, TimestampType, BinaryType, IntegerType, DateType
from datetime import datetime, timedelta

from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number
   

In [0]:
#synapse 
spark.conf.set('fs.azure.account.key.gdwpocdls.dfs.core.windows.net', dbutils.secrets.get(scope = 'BIFC-asagdwpocws-etl', key = 'StorageAccount-gdwpocdls-AccessKey'));
_url = dbutils.secrets.get(scope = 'BIFC-asagdwpocws-etl', key = 'Synapse-asagdwpocetl-ConnectionString');


In [0]:
df = parse_data(None,None,159,1,None,None,True)

display(df)

xpath(file_content,'/Data/Products/Product/EndPoint/Firewall/@disable')
xpath(file_content,'/Data/Products/Product/EndPoint/Firewall/@enable')
xpath(file_content,'/Data/Products/Product/EndPoint/ScanMode/@Convention')
xpath(file_content,'/Data/Products/Product/EndPoint/ScanMode/@SmartScan')
xpath(file_content,'/Data/Products/Product/EndPoint/CPU_Type/@CPU_32bit')
xpath(file_content,'/Data/Products/Product/EndPoint/CPU_Type/@CPU_64bit')
xpath(file_content,'/Data/Products/Product/EndPoint/CiscoNAC/@disable')
xpath(file_content,'/Data/Products/Product/EndPoint/CiscoNAC/@enable')

file_id,file_date,column_value,category,column_name
1086684193,2022-01-20,5,CPU_Type,32_Bit
1086685978,2022-01-20,1,CPU_Type,32_Bit
1086684090,2022-01-20,1,CPU_Type,32_Bit
1086656686,2022-01-20,3,CPU_Type,32_Bit
1086685949,2022-01-20,0,CPU_Type,32_Bit
1086685964,2022-01-20,4,CPU_Type,32_Bit
1086684356,2022-01-20,39,CPU_Type,32_Bit
1086685969,2022-01-20,1,CPU_Type,32_Bit
1086657144,2022-01-20,0,CPU_Type,32_Bit
1086639784,2022-01-20,2,CPU_Type,32_Bit


In [0]:
df = parse_data(None,None,159,2,None,None,True)

display(df)

xpath(file_content,'/Data/Products/Product/EndPoint/Firewall/@disable')

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-1096673393792987> in <module> 
 ----> 1 df = parse_data ( None , None , 159 , 2 , None , None , True ) 
 2 
 3 display ( df ) 

 <command-2456132037093057> in parse_data (project_name, group_name, project_group_id, result_type, startdate, enddata, with_file_date) 
 117 
 118 elif result_type == 2 : 
 --> 119 column_name = r . column_name . replace ( '.' , '_' ) 
 120 if isfristcol == True : 
 121 sqlscript = f"""select file_id,{with_file_date_col}{sql_part_2} as {column_name} from TBL_DATA_RAW""" 

 AttributeError : 'NoneType' object has no attribute 'replace'

In [0]:
from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number
    
def parse_data(project_name, group_name, project_group_id, result_type, startdate, enddata, with_file_date):

#     project_name = ''
#     group_name = ''
#     project_group_id = 140
#     result_type = 2
#     with_file_date = True

  
    if project_group_id == None:
        df_group = (spark.read
            .format('com.databricks.spark.sqldw')
            .option('url', _url)
            .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
            .option('useAzureMSI', 'true')
            .option('dbTable', 'BIFC_ETL.TBL_PROJECT_GROUP')
            .load()
         .filter(F.col('Project_Name') == project_name)
         .filter(F.col('Group_Name') == group_name)
         .select('Project_Group_id')
        )
        project_group_id = df_group.collect()[0][0]

    df_parse_config = (spark.read
            .format('com.databricks.spark.sqldw')
                    .option('url', _url)
                    .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                    .option('useAzureMSI', 'true')
                    .option('dbTable', 'BIFC_ETL.TBL_XML_PARSER_CONFIG')
                    .load()
                    .withColumn("sort",F.col("sort").cast(IntegerType()))
                    .filter(f'project_group_id=={project_group_id}')
                   )

    df_raw_config = (spark.read
                        .format('com.databricks.spark.sqldw')
                        .option('url', _url)
                        .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                        .option('useAzureMSI', 'true')
                        .option('dbTable', 'BIFC_ETL.TBL_XML_RAW_CONFIG')
                        .load()
                       )


    df_config_table = (df_parse_config.alias('m')
                      .filter(f'project_group_id=={project_group_id}')
                      .join(df_raw_config.alias('r'), [F.col('m.xpath_id')==F.col('r.xpath_id')], 'inner')
                      .drop(df_raw_config.xpath_id)
                      )

    df_config_table_list = df_config_table.select('source','xml_version','category').collect()

    source = df_config_table_list[0][0]
    xml_version =  df_config_table_list[0][1]
    category =  df_config_table_list[0][2]

    # df_config_table_list = df_config_table.filter("master_detail = 'D'").orderBy(F.col("level").desc()).select('raw_group_root','level').collect()
    # root_node = df_config_table_list[0][0]
    # level =  df_config_table_list[0][1]
    df_raw = (spark.read
          .format('com.databricks.spark.sqldw')
          .option('url', _url)
          .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
          .option('useAzureMSI', 'true')
          .option('dbTable', 'BIFC.TBL_DATA_RAW')
          .load()
          .filter(F.col('source') == source)
          .filter(F.col('xml_version') == xml_version )
          .filter(F.col('file_date') > startdate)
          .filter(F.col('file_date') < enddata)
          .withColumn('file_date', F.date_format(F.col('file_date'),"yyyy/MM/dd"))
          )

    df_raw.createOrReplaceTempView('TBL_DATA_RAW')
   
    
    if category != None and result_type == 2:
        df_t_col = (df_config_table
                    .withColumn('category', F.lit(''))
                    .withColumn('column_name', F.when(F.col('category').isNotNull(), F.concat(F.col('category'),F.lit('_'),F.col('column_name'))).otherwise(F.col('column_name')))
                    .groupBy('category','column_name','xpath_id').agg(F.min('sort').alias('sort'))
                    .orderBy(F.col('sort').asc())
                        )
    else:
        df_t_col = (df_config_table
                   .withColumn('category', F.when(F.col('category').isNull(), F.lit('')).otherwise(F.col('category')))
                   .groupBy('category','column_name','xpath_id').agg(F.min('sort').alias('sort'))
                   .orderBy(F.col('sort').asc())
                        )
            
    #debug   
    display(df_t_col)          
    df_t_col = (df_t_col.alias('r').join(df_config_table.alias('c'),  [F.col('r.xpath_id')==F.col('c.xpath_id')], 'inner')
               .drop('c.xpath_id,c.category,c.column_name,c.sort')
               .orderBy(F.col('r.sort').asc())
               )
    #debug
#     display(df_t_col)

    #generate xpath script , result_type == 1, concat row.  result_type == 2 concat column
    df_t_col_list = df_t_col.collect()

    #init
    with_file_date_col = 'file_date,' if with_file_date==True else ''
    dfs = [] # for type 1
    
    isfristcol = True # for type 2
    w = Window.orderBy(monotonically_increasing_id()) # for type 2

    for r in df_t_col_list:
        xpath = r.xpath 
        xpath_list = xpath.split('/')
        if r.master_detail == 'D':
            if r.xpath == r.raw_group_root:
                xpath_full_text = '/'.join(xpath_list[:-1])+'/'+xpath_list[-1].replace('@','') +'/text()' 
                sql_part_2 = f"xpath(file_content,'{xpath_full_text}')"
            else:
                xpath_full = '/'.join(xpath_list[:-1])+'/@'+xpath_list[-1].replace('@','')  
                sql_part_2 = f"xpath(file_content,'{xpath_full}')"  
        elif (r.master_detail == 'M') :

            xpath_full = '/'.join(xpath_list[:-1])+'/@'+xpath_list[-1].replace('@','') 
            sql_part_2 = f"xpath(file_content,'{xpath_full}')"
        #debug 
        print(sql_part_2)
        
        if result_type == 1:
            sqlscript = f"""select file_id,{with_file_date_col} {sql_part_2} as column_value from TBL_DATA_RAW"""
            _df = ( spark.sql(sqlscript)
                 .withColumn('category',F.lit(r.category))
                 .withColumn('column_value', F.explode('column_value'))
                 .withColumn('column_name',F.lit(r.column_name))
                
                  )
            dfs.append(_df)     
        elif result_type == 2:
            column_name = r.column_name.replace('.','_')
            if isfristcol == True:
                sqlscript = f"""select file_id,{with_file_date_col}{sql_part_2} as {column_name} from TBL_DATA_RAW"""
                parse_result =( spark.sql(sqlscript)
                     .withColumn(column_name, F.explode(column_name))
                      .withColumn("columnindex", row_number().over(w))
                      )
                isfristcol = False
            else:
                sqlscript = f"""select file_id, {sql_part_2} as {column_name} from TBL_DATA_RAW"""
                _df =( spark.sql(sqlscript)
                     .select(F.explode(column_name).alias(column_name))
                     .withColumn("columnindex", row_number().over(w))
                     )
                parse_result = parse_result.join(_df, parse_result.columnindex == _df.columnindex, 'inner').drop(_df.columnindex)

    #output result df type1 unionall / type2 join all column
    
        
    if with_file_date==True:
        collist = ['file_id','file_date','category','column_name','column_value']
    else:
         collist = ['file_id','category','column_name','column_value']
    
    if result_type == 1:
        parse_result = reduce(DataFrame.unionAll, dfs).select([col for col in collist])
    elif result_type == 2:
        parse_result = parse_result.drop(parse_result.columnindex)

    return parse_result
